In [ ]:
import pandas as pd
import requests
import time
from openpyxl import Workbook, load_workbook
from datetime import datetime
from pathlib import Path

def get_stock_data(stock_code):
    """获取单只股票的看涨数据"""
    prefix = 'sh' if stock_code.startswith(('6', '5', '9')) else 'sz'
    url = f'https://eminterservice.eastmoney.com/UserData/GetWebTape?code={prefix}{stock_code}'
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36 Edg/136.0.0.0'}
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            json_data = response.json()
            return float(json_data['Data']['TapeZ'])
    except Exception as e:
        print(f"获取 {stock_code} 数据失败: {str(e)}")
    return None

def init_workbook(output_file):
    """初始化或加载Excel工作簿"""
    output_path = Path(output_file)
    if output_path.exists():
        try:
            return load_workbook(output_file, read_only=False)
        except Exception as e:
            print(f"加载工作簿失败，将创建新文件。错误详情：{str(e)}")
            return Workbook()
    else:
        wb = Workbook()
        if 'Sheet' in wb.sheetnames:
            del wb['Sheet']
        return wb

def is_valid_excel(file_path):
    """验证是否为有效Excel文件"""
    try:
        pd.read_excel(file_path, nrows=1)
        return True
    except:
        return False

def process_index_data(input_file, output_file):
    """动态处理存在的指数Sheet"""
    if not is_valid_excel(input_file):
        print("错误：输入文件格式无效")
        return
    
    xls = pd.ExcelFile(input_file)
    available_sheets = xls.sheet_names
    target_indexes = ['上证50', '沪深300', '中证500', '中证1000', '中证2000']
    valid_indexes = [name for name in target_indexes if name in available_sheets]
    
    if not valid_indexes:
        print("错误：输入文件中未找到任何目标指数Sheet")
        return
    
    print(f"检测到有效Sheet: {valid_indexes}")
    current_date = datetime.now().strftime('%Y-%m-%d')
    wb = init_workbook(output_file)
    
    for index_name in valid_indexes:
        print(f"\n正在处理 [{index_name}]...")
        
        try:
            df = pd.read_excel(xls, sheet_name=index_name)
            df = df.rename(columns={
                '成份股代码': '成分股代码',
                '成份股名称': '成分股名称'
            })
            codes = df['成分股代码'].astype(str).str.zfill(6).tolist()
            names = df['成分股名称'].tolist()
        except KeyError as e:
            print(f"跳过 [{index_name}]：缺少列 {str(e)}")
            continue
        except Exception as e:
            print(f"跳过 [{index_name}]：读取失败 - {str(e)}")
            continue
        
        stock_headers = [f"{code} {name}" for code, name in zip(codes, names)]
        today_data = {'日期': current_date}
        
        for idx, (code, name) in enumerate(zip(codes, names), 1):
            zhang = get_stock_data(code)
            today_data[f"{code} {name}"] = f"{zhang}%" if zhang else "N/A"
            print(f"进度：{index_name} ({idx}/{len(codes)}) - {code} {name}")
            time.sleep(0.3)
        
        sheet_exists = index_name in wb.sheetnames
        if sheet_exists:
            ws = wb[index_name]
            existing_headers = [cell.value for cell in ws[1]]
            if existing_headers != ['日期'] + stock_headers:
                print(f"{index_name} 列标题不匹配，跳过更新")
                continue
        else:
            ws = wb.create_sheet(title=index_name)
            ws.append(['日期'] + stock_headers)
        
        ws.append([today_data.get(header, 'N/A') for header in ['日期'] + stock_headers])
        wb.save(output_file)
        print(f"成功更新 [{index_name}]")
    
    wb.close()
    print(f"\n处理完成，文件路径：{output_file}")

if __name__ == "__main__":
    input_path = r'C:\Users\admin\Desktop\index_data(1).xlsx'
    
    if not Path(input_path).exists():
        print(f"错误：输入文件不存在 - {input_path}")
        exit()
        
    process_index_data(
        input_file=input_path,
        output_file='stock_sentiment_data.xlsx'
    )  